In [1]:
import numpy as np

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf


In [5]:
path_to_file = "shakespeare.txt"


In [7]:
text = open(path_to_file,'r').read()


In [8]:
vocab = sorted(set(text))

In [12]:
char_to_ind = {char : ind for ind,char in enumerate(vocab)}

In [13]:
ind_to_char = np.array(vocab)

In [14]:
ind_to_char[33]

np.str_('H')

In [15]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [19]:
sample = text[:500]

In [20]:
new_sample = encoded_text[:500]

In [21]:
new_sample

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

In [22]:
seq_len = 120

In [23]:
total_num_seq = len(text) // (seq_len+1)

In [24]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [26]:
char_dataset.batch

<bound method DatasetV2.batch of <_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>>

In [27]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [31]:
def create_seq_targets(seq):
    input_txt = seq[:-1] #Hello my nam
    target_txt = seq[1:] #ello my name
    return input_txt,target_txt

In [32]:
dataset = sequences.map(create_seq_targets)

In [34]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


2025-05-07 22:18:13.334709: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [35]:
batch_size = 128

In [39]:
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder = True)

In [40]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

In [41]:
vocab_size =len(vocab)

In [42]:
embed_dim = 64

In [43]:
rnn_neurons = 1026

In [44]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [55]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [45]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import  Embedding,GRU, Dense

In [72]:

def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    model = Sequential()

    #maps each input to a vector
    model.add(Embedding(vocab_size,embed_dim,input_shape=[seq_len,None]))

    #recurrent neural network for series prediction
    model.add(GRU(rnn_neurons,return_sequences= True
    ,stateful = True, recurrent_initializer = 'glorot_uniform'))

    #converts output to 128 neuron
    model.add(Dense(vocab_size))

    model.compile('adam',loss = sparse_cat_loss)
    return model

In [73]:
model = create_model(vocab_size = vocab_size,embed_dim = embed_dim,
                    rnn_neurons = rnn_neurons,batch_size = batch_size)

ValueError: Input 0 of layer "gru_10" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 120, None, 64)

In [57]:
model = create_model(vocab_size=vocab_size,embed_dim=embed_dim,rnn_neurons=rnn_neurons,batch_size=batch_size)

In [58]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)